In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [4]:
Rebuild_data=True

In [5]:
class DogsVsCats:
    img_size=50
    path='/kaggle/input/'
    cats='kaggle/input/dogs-cats-images/dataset/training_set/cats'
    dogs='kaggle/input/dogs-cats-images/dataset/training_set/dogs'
    labels={cats:0,dogs:1}
    training_data=[]
    catcount=0
    dogcount=0
    
    def make_training_data(self):
        for dirname, _, filenames in os.walk('/kaggle/input'):
            for filename in filenames:
                #print(filename)
                path=os.path.join(dirname, filename)
                img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img=cv2.resize(img, (self.img_size,self.img_size))
                print(path)
                break

In [ ]:
if Rebuild_data:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()

In [23]:
class DogsVsCats:
    img_size=50
    cats='/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/cats/'
    dogs='/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/dogs/'
    labels={cats:0,dogs:1}
    training_data=[]
    catcount=0
    dogcount=0
                
    def make_training_data(self):
        for label in self.labels:
            try:
                for f in tqdm(os.listdir(label)):
                    path=os.path.join(label,f)
                    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img=cv2.resize(img,(self.img_size,self.img_size))
                    self.training_data.append([np.array(img),np.eye(2)[self.labels[label]]])
                    
                    if label == self.cats:
                        self.catcount+=1
                    elif label == self.dogs:
                        self.dogcount+=1
            except Exception as e:
                pass  
            
        np.random.shuffle(self.training_data)
        np.save('training_data.npy',self.training_data)
        print('Cats:',self.catcount)
        print('Dogs:',self.dogcount)

In [21]:
if Rebuild_data:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()